# Massachusetts School Data Investigation
The following is our investigation into the 2018-19 education data for Massachusetts. All of the data, except the NERDS dataset, has been sourced by the DOE website [here](https://profiles.doe.mass.edu/state_report/).
### Alex Craig & Aaron Xie

## 0. Load Packages & Imports

In [529]:
import pandas as pd
import re

## 0.1 Initial Data Loading

In [530]:
nerds_df = pd.read_excel("../data/MA_1819_final_January_28th_22.xlsx")
nerds_df.head()

,state,year,distid_stateassigned,schoolid_stateassigned,distname,schoolname,ncesdistid_admin,ncesdistid_geo,census_id,ncesid,...,pp_PD_cent_fed_raw_MA,pp_instrumaterial_cent_fed_raw_MA,pp_guidepsych_cent_fed_raw_MA,countenroll,noaudit,nositecen,adfinvars,distflex,flag_nerds,flag_f33
0,MA,2018-2019,445,105,Abby Kelley Foster Charter Public School,Abby Kelley Foster Charter Public School,2500051,2500051,NaN,250005101247,...,0,0,0,0,0,0,1,0,0,0
1,MA,2018-2019,1,1,Abington,Abington Early Education Program,2501650,2501650,2.230120e+13,250165002727,...,0,0,0,0,0,0,1,0,0,0
2,MA,2018-2019,1,505,Abington,Abington High,2501650,2501650,2.230120e+13,250165000001,...,0,0,0,0,0,0,1,0,0,0
3,MA,2018-2019,1,405,Abington,Abington Middle School,2501650,2501650,2.230120e+13,250165000002,...,0,0,0,0,0,0,1,0,0,0
4,MA,2018-2019,1,20,Abington,Beaver Brook Elementary,2501650,2501650,2.230120e+13,250165000434,...,0,0,0,0,0,0,1,0,0,0


In [531]:
mcas_df = pd.read_excel("../data/mcas_results_2019.xlsx")
mcas_df.head()

,School Name,School Code,Subject,M+E #,M+E %,E #,E %,M #,M %,PM #,PM %,NM #,NM %,No. of Students Included,Avg. Scaled Score,SGP,Included In SGP,Ach. PCTL
0,Abby Kelley Foster Charter Public (District) -...,4450105,ELA,346,49,37,5,309,44,303,43,50,7,699,499.4,51.4,578,38.0
1,Abby Kelley Foster Charter Public (District) -...,4450105,MATH,282,40,23,3,259,37,357,51,60,9,699,495.9,51.3,579,35.0
2,Abby Kelley Foster Charter Public (District) -...,4450105,SCI,80,34,4,2,76,32,135,57,21,9,236,492.6,NaN,NaN,27.0
3,Abington - Abington Middle School,10405,ELA,351,54,51,8,300,46,257,39,46,7,654,501.8,46.6,634,47.0
4,Abington - Abington Middle School,10405,MATH,294,45,29,4,265,40,308,47,53,8,655,497.4,44.2,634,41.0


In [532]:
enrollment_demographics_df = pd.read_excel("../data/enrollment_demographics_2019.xlsx")
enrollment_demographics_df.head()

,School Name,School Code,African American,Asian,Hispanic,White,Native American,"Native Hawaiian, Pacific Islander","Multi-Race, Non-Hispanic",Males,Females
0,Abby Kelley Foster Charter Public (District) -...,4450105,51.3,3.4,22.7,19.7,0.2,0.1,2.5,47.5,52.5
1,Abington - Abington Early Education Program,10001,10.0,1.1,10.0,78.9,0.0,0.0,0.0,50.0,50.0
2,Abington - Abington High,10505,3.9,2.4,7.9,85.0,0.0,0.2,0.7,49.0,51.0
3,Abington - Abington Middle School,10405,2.1,1.8,7.1,87.4,0.4,0.0,1.2,52.4,47.6
4,Abington - Beaver Brook Elementary,10020,4.8,2.4,7.3,84.0,1.1,0.0,0.4,50.5,49.5


In [533]:
grad_rates_df = pd.read_excel("../data/graduation_rates_2019.xlsx")
grad_rates_df.head()

,School Name,School Code,# in Cohort,% Graduated,% Still in School,% Non-Grad Completers,% H.S. Equiv.,% Dropped Out,% Permanently Excluded
0,Abby Kelley Foster Charter Public (District) -...,4450105,84,97.6,0.0,0.0,0.0,2.4,0.0
1,Abington - Abington High,10505,115,91.3,1.7,0.9,0.0,6.1,0.0
2,Academy Of the Pacific Rim Charter Public (Dis...,4120530,40,95.0,5.0,0.0,0.0,0.0,0.0
3,Acton-Boxborough - Acton-Boxborough Regional High,6000505,493,97.8,1.2,0.0,0.2,0.8,0.0
4,Adams-Cheshire - Hoosac Valley High School,6030505,88,89.8,2.3,1.1,2.3,4.5,0.0


In [534]:
socio_eco_df = pd.read_excel("../data/selectedpopulations_2019.xlsx")
socio_eco_df.head()

,School Name,School Code,First Language Not English #,First Language Not English %,English Language Learner #,English Language Learner %,Students With Disabilities #,Students With Disabilities %,Low Income #,Low Income %,Free Lunch #,Free Lunch %,Reduced Lunch #,Reduced Lunch %,High Needs #,High Needs #.1,Economically Disadvantaged #,Economically Disadvantaged %
0,Abby Kelley Foster Charter Public (District) -...,4450105,910,63.7,141,9.9,171,12.0,,,,,,,803,56.2,538,37.7
1,Abington - Abington Early Education Program,10001,9,10.0,2,2.2,34,37.8,,,,,,,46,51.1,22,24.4
2,Abington - Abington High,10505,37,6.8,10,1.8,69,12.7,,,,,,,161,29.5,105,19.3
3,Abington - Abington Middle School,10405,67,10.0,25,3.7,91,13.5,,,,,,,243,36.2,164,24.4
4,Abington - Beaver Brook Elementary,10020,68,14.9,44,9.7,65,14.3,,,,,,,158,34.7,89,19.6


In [535]:
staff_demographics_df = pd.read_excel("../data/staff_demographics_2019.xlsx")
staff_demographics_df.head()

,District/School Name,District/School Code,African American (%),Asian (%),Hispanic (%),White (%),Native American (%),"Native Hawaiian, Pacific Islander (%)","Multi-Race,Non-Hispanic (%)",Females (%),Males (%),FTE Count
0,Abby Kelley Foster Charter Public (District) -...,4450105,3.7,0.9,1.8,92.4,0.0,0.0,1.2,79.2,20.8,163.0
1,Abington - Abington Early Education Program,10001,0.0,0.0,0.0,100.0,0.0,0.0,0.0,100.0,0.0,16.0
2,Abington - Abington High,10505,0.0,1.1,1.6,95.7,0.0,0.0,1.6,72.5,27.5,62.4
3,Abington - Abington Middle School,10405,2.7,0.0,4.8,92.5,0.0,0.0,0.0,77.3,22.7,73.7
4,Abington - Beaver Brook Elementary,10020,0.0,0.0,1.7,98.3,0.0,0.0,0.0,99.3,0.7,57.7


In [536]:
student_to_teacher_df = pd.read_excel("../data/teacherdata_2019.xlsx")
student_to_teacher_df.head()

,School Name,School Code,Total # of Teachers (FTE),% of Teachers Licensed,Student / Teacher Ratio,Percent of Experienced Teachers,Percent of Teachers without Waiver or Provisional License,Percent Teaching In-Field
0,Abby Kelley Foster Charter Public (District) -...,4450105,116.4,94.5,12.3 to 1,76.4,87.1,85.1
1,Abington - Abington Early Education Program,10001,4.0,100.0,22.5 to 1,100.0,100.0,100.0
2,Abington - Abington High,10505,37.9,100.0,14.4 to 1,92.6,88.9,83.3
3,Abington - Abington Middle School,10405,38.2,100.0,17.6 to 1,87.2,95.0,93.0
4,Abington - Beaver Brook Elementary,10020,25.7,100.0,17.7 to 1,84.5,96.1,100.0


In [537]:
teacher_salary_df = pd.read_excel("../data/teachersalaries_2019.xlsx")
teacher_salary_df.head()

,District Name,District Code,Salary Totals,Average Salary,FTE Count
0,Abington,10000,"$11,469,788","$93,861",122.2
1,Acton-Boxborough,6000000,"$32,945,978","$86,677",380.1
2,Acushnet,30000,"$5,217,686","$69,108",75.5
3,Adams-Cheshire,6030000,"$6,920,007","$84,212",82.2
4,Agawam,50000,"$25,584,999","$79,828",320.5


## 0.2 Cleaning Data

### 0.2.1 Cleaning the NERDS Data
Lets clean the Nerds dataframe by only keeping the columns that we want, as it currently has many columns, most of which we don't need.

Columns for future Merges:
- `distname`: District Name
- `schoolname`: School Name

Columns for Spending Per Pupil:
- `pp_stloc`: State and Local Spending Per Pupil
- `pp_fed`: Federal Spending Per Pupil
- `pp_total`: Total Spending Per Pupil

Columns for Total Spending Per Pupil:
- `schoolstloc`: State and Local Spending
- `schoolfed`: Federal Spending
- `schooltot`: Total Spending

Columns for Enrollment:
- `ncesenroll`: Enrollment
- `gradespan`: Grades Spanned

Columns for Flagging:
- `flag_nerds`: Flag for suspicious data on school
- `flag_f33`: Flag for suspicious data on district

For this investigation, since we are not comparing across states, we are opting to use to `_raw` instead of the `_norm_nerds` columns which contain values which have been adjusted for state differences.

The information about these columns was found in the [NERDS Documentation](https://georgetown.app.box.com/s/nyt1ckphiblaj4tyuoigtri8bn38kk85/file/1080532555684).

In [538]:
nerds_df_clean = nerds_df[['distname', 'schoolname', 'flag_nerds', 'flag_f33', 'ncesenroll', 'gradespan', 'pp_stloc_raw_MA', 'pp_fed_raw_MA', 'pp_total_raw_MA', 'schoolstloc_raw_MA', 'schoolfed_raw_MA', 'schooltot_raw_MA']]

# Let's also make district and school names lowercase for easier merging
nerds_df_clean["distname_lower"] = nerds_df_clean["distname"].str.lower()
nerds_df_clean["schoolname_lower"] = nerds_df_clean["schoolname"].str.lower()

nerds_df_clean.head()

/var/folders/1y/tx5xv7mx325cz5z1q7xc35sm0000gn/T/ipykernel_24236/3621529377.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nerds_df_clean["distname_lower"] = nerds_df_clean["distname"].str.lower()
/var/folders/1y/tx5xv7mx325cz5z1q7xc35sm0000gn/T/ipykernel_24236/3621529377.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nerds_df_clean["schoolname_lower"] = nerds_df_clean["schoolname"].str.lower()


,distname,schoolname,flag_nerds,flag_f33,ncesenroll,gradespan,pp_stloc_raw_MA,pp_fed_raw_MA,pp_total_raw_MA,schoolstloc_raw_MA,schoolfed_raw_MA,schooltot_raw_MA,distname_lower,schoolname_lower
0,Abby Kelley Foster Charter Public School,Abby Kelley Foster Charter Public School,0,0,1428.0,KG-12,14354.612514,1006.982538,15361.595053,20461351.77,1435373.05,21896724.82,abby kelley foster charter public school,abby kelley foster charter public school
1,Abington,Abington Early Education Program,0,0,90.0,PK-PK,14028.783797,961.863052,14990.646848,1444029.478804,99007.770134,1543037.248938,abington,abington early education program
2,Abington,Abington High,0,0,545.0,09-12,14880.475672,20.812074,14901.287746,8052652.079188,11262.569511,8063914.648699,abington,abington high
3,Abington,Abington Middle School,0,0,672.0,05-08,13057.825084,83.417661,13141.242745,8839857.407697,56471.903107,8896329.310804,abington,abington middle school
4,Abington,Beaver Brook Elementary,0,0,455.0,KG-02,13134.592802,1105.059138,14239.65194,5944497.792223,500131.348068,6444629.140291,abington,beaver brook elementary


Let's make lower case columns for `distname` and `schoolname` for easier merging later.

In [539]:
nerds_df_clean["distname_lower"] = nerds_df_clean["distname"].str.lower()
nerds_df_clean["schoolname_lower"] = nerds_df_clean["schoolname"].str.lower()

nerds_df_clean.head()

/var/folders/1y/tx5xv7mx325cz5z1q7xc35sm0000gn/T/ipykernel_24236/2373075718.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nerds_df_clean["distname_lower"] = nerds_df_clean["distname"].str.lower()
/var/folders/1y/tx5xv7mx325cz5z1q7xc35sm0000gn/T/ipykernel_24236/2373075718.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nerds_df_clean["schoolname_lower"] = nerds_df_clean["schoolname"].str.lower()


,distname,schoolname,flag_nerds,flag_f33,ncesenroll,gradespan,pp_stloc_raw_MA,pp_fed_raw_MA,pp_total_raw_MA,schoolstloc_raw_MA,schoolfed_raw_MA,schooltot_raw_MA,distname_lower,schoolname_lower
0,Abby Kelley Foster Charter Public School,Abby Kelley Foster Charter Public School,0,0,1428.0,KG-12,14354.612514,1006.982538,15361.595053,20461351.77,1435373.05,21896724.82,abby kelley foster charter public school,abby kelley foster charter public school
1,Abington,Abington Early Education Program,0,0,90.0,PK-PK,14028.783797,961.863052,14990.646848,1444029.478804,99007.770134,1543037.248938,abington,abington early education program
2,Abington,Abington High,0,0,545.0,09-12,14880.475672,20.812074,14901.287746,8052652.079188,11262.569511,8063914.648699,abington,abington high
3,Abington,Abington Middle School,0,0,672.0,05-08,13057.825084,83.417661,13141.242745,8839857.407697,56471.903107,8896329.310804,abington,abington middle school
4,Abington,Beaver Brook Elementary,0,0,455.0,KG-02,13134.592802,1105.059138,14239.65194,5944497.792223,500131.348068,6444629.140291,abington,beaver brook elementary


### 0.2.2 Cleaning the MCAS Data
MCAS is the standardized test which Massachusetts uses for all grade levels through high school. Let's convert the dataframe to a wide format so that each District is a single row, and each column is a different subject. We will also keep the `School Name` and `School Code` columns for future merges.

In [540]:
mcas_df_clean = pd.pivot(mcas_df, index=['School Name', 'School Code'], columns='Subject', values='Avg. Scaled Score').reset_index()
# mcas_df_clean.columns = ["distname", "distcode", "ela_avg_scaled_score", "math_avg_scaled_score", "sci_avg_scaled_score"]

mcas_df_clean.head()

Subject,School Name,School Code,ELA,MATH,SCI
0,Abby Kelley Foster Charter Public (District) -...,4450105,499.4,495.9,492.6
1,Abington - Abington Middle School,10405,501.8,497.4,496.8
2,Abington - Woodsdale Elementary School,10015,508.5,505.1,NaN
3,Academy Of the Pacific Rim Charter Public (Dis...,4120530,496.5,491.4,491.6
4,Acton-Boxborough - Blanchard Memorial School,6000005,514.7,513.8,521.2


We see that `School Name` contains both school and district names. Let's split them up, convert to lower case, and also remove the "(District)" from the end of the district name. Also, we will make this process a function to use for the cleaning of other Massachusetts data. 

In [541]:
split_names_test = [name.split(" - ") for name in mcas_df_clean["School Name"]]
print([name for name in split_names_test if len(name) != 2])

[['Community Day Charter Public School', 'Gateway (District)', 'Community Day Charter Public School', 'Gateway'], ['Community Day Charter Public School', 'R. Kingman Webster (District)', 'Community Day Charter Public School', 'R. Kingman Webster']]


We see a few schools with names that are not in the format 'District - School'. We will have to write some logic to handle these cases. We can see that there are two cases, and in both of them school comes first, district comes second, and the length of their list is greater than 2.

In [542]:
def clean_school_name(dataframe, column_name):
    split_names = [name.split(" - ") for name in dataframe[column_name]]
    
    dist_names_lower = [name[0].lower() if len(name) == 2 else name[1].lower() for name in split_names]
    school_names_lower = [name[1].lower() if len(name) == 2 else name[0].lower() for name in split_names]

    regex = r"\s*\(\s*district\s*\)\s*"
    dist_names_lower_clean = [re.sub(regex, "", name) if name is not None else None for name in dist_names_lower]

    dataframe["distname_lower"] = dist_names_lower_clean
    dataframe["schoolname_lower"] = school_names_lower

In [543]:
clean_school_name(mcas_df_clean, "School Name")

Next, let's make a `avg_score` column which is the average of the 3 MCAS scores. There are a few NaN scores, however these should still work with the `mean` function.

In [544]:
avg_score = mcas_df_clean[['ELA', 'MATH', 'SCI']].mean(axis=1)
mcas_df_clean["avg_score"] = avg_score

mcas_df_clean.head()

Subject,School Name,School Code,ELA,MATH,SCI,distname_lower,schoolname_lower,avg_score
0,Abby Kelley Foster Charter Public (District) -...,4450105,499.4,495.9,492.6,abby kelley foster charter public,abby kelley foster charter public school,495.966667
1,Abington - Abington Middle School,10405,501.8,497.4,496.8,abington,abington middle school,498.666667
2,Abington - Woodsdale Elementary School,10015,508.5,505.1,NaN,abington,woodsdale elementary school,506.800000
3,Academy Of the Pacific Rim Charter Public (Dis...,4120530,496.5,491.4,491.6,academy of the pacific rim charter public,academy of the pacific rim charter public school,493.166667
4,Acton-Boxborough - Blanchard Memorial School,6000005,514.7,513.8,521.2,acton-boxborough,blanchard memorial school,516.566667


### 0.2.3 Cleaning the Enrollment Demographics Data
The `enrollment_demographics_df` comes from the same source as the `mcas_df`, which is Massachusett's Department of Education, so it is formatted in the same way and we just need to clean the `School Name` column.

In [545]:
enrollment_demographics_df_clean = enrollment_demographics_df.copy()
clean_school_name(enrollment_demographics_df_clean, "School Name")

enrollment_demographics_df_clean.head()

,School Name,School Code,African American,Asian,Hispanic,White,Native American,"Native Hawaiian, Pacific Islander","Multi-Race, Non-Hispanic",Males,Females,distname_lower,schoolname_lower
0,Abby Kelley Foster Charter Public (District) -...,4450105,51.3,3.4,22.7,19.7,0.2,0.1,2.5,47.5,52.5,abby kelley foster charter public,abby kelley foster charter public school
1,Abington - Abington Early Education Program,10001,10.0,1.1,10.0,78.9,0.0,0.0,0.0,50.0,50.0,abington,abington early education program
2,Abington - Abington High,10505,3.9,2.4,7.9,85.0,0.0,0.2,0.7,49.0,51.0,abington,abington high
3,Abington - Abington Middle School,10405,2.1,1.8,7.1,87.4,0.4,0.0,1.2,52.4,47.6,abington,abington middle school
4,Abington - Beaver Brook Elementary,10020,4.8,2.4,7.3,84.0,1.1,0.0,0.4,50.5,49.5,abington,beaver brook elementary


### 0.2.4 Cleaning the Graduation Rates Data
The `grad_rates_df` also comes from Massachusett's Department of Education, so we just need to clean the `School Name` column in the same way.

In [546]:
grad_rates_df_clean = grad_rates_df.copy()
clean_school_name(grad_rates_df_clean, "School Name")

grad_rates_df_clean.head()

,School Name,School Code,# in Cohort,% Graduated,% Still in School,% Non-Grad Completers,% H.S. Equiv.,% Dropped Out,% Permanently Excluded,distname_lower,schoolname_lower
0,Abby Kelley Foster Charter Public (District) -...,4450105,84,97.6,0.0,0.0,0.0,2.4,0.0,abby kelley foster charter public,abby kelley foster charter public school
1,Abington - Abington High,10505,115,91.3,1.7,0.9,0.0,6.1,0.0,abington,abington high
2,Academy Of the Pacific Rim Charter Public (Dis...,4120530,40,95.0,5.0,0.0,0.0,0.0,0.0,academy of the pacific rim charter public,academy of the pacific rim charter public school
3,Acton-Boxborough - Acton-Boxborough Regional High,6000505,493,97.8,1.2,0.0,0.2,0.8,0.0,acton-boxborough,acton-boxborough regional high
4,Adams-Cheshire - Hoosac Valley High School,6030505,88,89.8,2.3,1.1,2.3,4.5,0.0,adams-cheshire,hoosac valley high school


### 0.2.5 Cleaning the Socio-Economic Data
The `socio_eco_df` also comes from Massachusett's Department of Education, so we just need to clean the `School Name` column in the same way.

In [563]:
socio_eco_df_clean = socio_eco_df.copy()
clean_school_name(socio_eco_df_clean, "School Name")

socio_eco_df_clean.head()

,School Name,School Code,First Language Not English #,First Language Not English %,English Language Learner #,English Language Learner %,Students With Disabilities #,Students With Disabilities %,Low Income #,Low Income %,Free Lunch #,Free Lunch %,Reduced Lunch #,Reduced Lunch %,High Needs #,High Needs #.1,Economically Disadvantaged #,Economically Disadvantaged %,distname_lower,schoolname_lower
0,Abby Kelley Foster Charter Public (District) -...,4450105,910,63.7,141,9.9,171,12.0,,,,,,,803,56.2,538,37.7,abby kelley foster charter public,abby kelley foster charter public school
1,Abington - Abington Early Education Program,10001,9,10.0,2,2.2,34,37.8,,,,,,,46,51.1,22,24.4,abington,abington early education program
2,Abington - Abington High,10505,37,6.8,10,1.8,69,12.7,,,,,,,161,29.5,105,19.3,abington,abington high
3,Abington - Abington Middle School,10405,67,10.0,25,3.7,91,13.5,,,,,,,243,36.2,164,24.4,abington,abington middle school
4,Abington - Beaver Brook Elementary,10020,68,14.9,44,9.7,65,14.3,,,,,,,158,34.7,89,19.6,abington,beaver brook elementary


### 0.2.6 Cleaning the Staff Demographics Data
The `staff_demographic_df` also comes from Massachusett's Department of Education, so we just need to clean the `School Name` column in the same way.

In [560]:
staff_demographics_df_clean = staff_demographics_df.copy()
clean_school_name(staff_demographics_df_clean, "District/School Name")

staff_demographics_df_clean.head()

,District/School Name,District/School Code,African American (%),Asian (%),Hispanic (%),White (%),Native American (%),"Native Hawaiian, Pacific Islander (%)","Multi-Race,Non-Hispanic (%)",Females (%),Males (%),FTE Count,distname_lower,schoolname_lower
0,Abby Kelley Foster Charter Public (District) -...,4450105,3.7,0.9,1.8,92.4,0.0,0.0,1.2,79.2,20.8,163.0,abby kelley foster charter public,abby kelley foster charter public school
1,Abington - Abington Early Education Program,10001,0.0,0.0,0.0,100.0,0.0,0.0,0.0,100.0,0.0,16.0,abington,abington early education program
2,Abington - Abington High,10505,0.0,1.1,1.6,95.7,0.0,0.0,1.6,72.5,27.5,62.4,abington,abington high
3,Abington - Abington Middle School,10405,2.7,0.0,4.8,92.5,0.0,0.0,0.0,77.3,22.7,73.7,abington,abington middle school
4,Abington - Beaver Brook Elementary,10020,0.0,0.0,1.7,98.3,0.0,0.0,0.0,99.3,0.7,57.7,abington,beaver brook elementary


### 0.2.7 Cleaning the Student to Teacher Data
The `student_to_teacher_df` also comes from Massachusett's Department of Education, so we just need to clean the `School Name` column in the same way. However, we are only interested in the `Student / Teacher Ratio` column, so we will only keep and clean that.

In [569]:
student_to_teacher_df_clean = student_to_teacher_df[["School Name", "Student / Teacher Ratio"]].copy()
clean_school_name(student_to_teacher_df_clean, "School Name")
student_to_teacher_df_clean.rename(columns={"Student / Teacher Ratio": "student_teacher_ratio"}, inplace=True)

split_ratio = [str(ratio).split(" to ") for ratio in student_to_teacher_df_clean["student_teacher_ratio"]]
student_to_teacher_df_clean["student_teacher_ratio"] = [float(ratio[0]) for ratio in split_ratio]

student_to_teacher_df_clean[student_to_teacher_df_clean["student_teacher_ratio"].isna()]

,School Name,student_teacher_ratio,distname_lower,schoolname_lower
276,Boston - West Roxbury Academy,NaN,boston,west roxbury academy
1482,Springfield - Gateway to College at Holyoke Co...,NaN,springfield,gateway to college at holyoke community college
1483,Springfield - Gateway to College at Springfiel...,NaN,springfield,gateway to college at springfield technical co...
1607,Uxbridge - Gateway to College,NaN,uxbridge,gateway to college


We see there are a few schools with `NaN` as the value to student_to_teacher_ratio. We will replace these with the mean of the column and flag them.

In [571]:
student_to_teacher_df_clean["nan_flag"] = student_to_teacher_df_clean["student_teacher_ratio"].isna()
avg_student_teacher_ratio = student_to_teacher_df_clean["student_teacher_ratio"].mean()

student_to_teacher_df_clean["student_teacher_ratio"] = student_to_teacher_df_clean["student_teacher_ratio"].fillna(avg_student_teacher_ratio)

student_to_teacher_df_clean[student_to_teacher_df_clean["nan_flag"]]

,School Name,student_teacher_ratio,distname_lower,schoolname_lower,nan_flag
276,Boston - West Roxbury Academy,13.646363,boston,west roxbury academy,True
1482,Springfield - Gateway to College at Holyoke Co...,13.646363,springfield,gateway to college at holyoke community college,True
1483,Springfield - Gateway to College at Springfiel...,13.646363,springfield,gateway to college at springfield technical co...,True
1607,Uxbridge - Gateway to College,13.646363,uxbridge,gateway to college,True


### 0.2.8 Cleaning the Teacher Salaries Data
Teacher salaries are only available by district, so we just need to convert the `District Name` column to lower case.

In [572]:
teacher_salary_df_clean = teacher_salary_df.copy()
teacher_salary_df_clean["distname_lower"] = teacher_salary_df_clean["District Name"].str.lower()

teacher_salary_df_clean.head()

,District Name,District Code,Salary Totals,Average Salary,FTE Count,distname_lower
0,Abington,10000,"$11,469,788","$93,861",122.2,abington
1,Acton-Boxborough,6000000,"$32,945,978","$86,677",380.1,acton-boxborough
2,Acushnet,30000,"$5,217,686","$69,108",75.5,acushnet
3,Adams-Cheshire,6030000,"$6,920,007","$84,212",82.2,adams-cheshire
4,Agawam,50000,"$25,584,999","$79,828",320.5,agawam
